1. 证明Transformer三角函数可以让模型感知到相对位置信息

回顾正弦位置编码的定义：

$PE_{(pos,2i)} = \sin\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right)$

$PE_{(pos,2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right)$

我们考虑一个固定的偏移k，想要找到一种变换M（可能与k有关），使得上述关系成立。

实际上，我们可以注意到，正弦和余弦函数具有以下性质：

```text
sin(α + β) = sin(α)cos(β) + cos(α)sin(β)
cos(α + β) = cos(α)cos(β) - sin(α)sin(β)
```
因此，对于每个频率分量（即每个i），我们有：

```text
令w_i = 1/(10000^(2i/d_model))。
PE(pos+k, 2i) = sin((pos+k) * w_i) = sin(pos*w_i + k*w_i) 
              = sin(pos*w_i)cos(k*w_i) + cos(pos*w_i)sin(k*w_i)
              = PE(pos,2i)*cos(k*w_i) + PE(pos,2i+1)*sin(k*w_i)

PE(pos+k, 2i+1) = cos((pos+k)*w_i) = cos(pos*w_i + k*w_i)
                = cos(pos*w_i)cos(k*w_i) - sin(pos*w_i)sin(k*w_i)
                = PE(pos,2i+1)*cos(k*w_i) - PE(pos,2i)*sin(k*w_i)

```

因此，我们可以将PE(pos+k)表示为PE(pos)的一个线性函数。具体来说，对于每个频率i，变换只涉及两个维度（2i和2i+1），并且这个变换是一个旋转矩阵：

```text
[ PE(pos+k, 2i)   ]   =   [ cos(k*w_i)   sin(k*w_i) ]   [ PE(pos, 2i)   ]
[ PE(pos+k, 2i+1) ]       [ -sin(k*w_i)  cos(k*w_i) ]   [ PE(pos, 2i+1) ]
```


注意，这个旋转矩阵只依赖于偏移k和频率w_i，而与具体的位置pos无关。

因此，对于整个向量PE(pos+k)，我们可以通过一个块对角矩阵（每个块对应一个频率的旋转矩阵）来从PE(pos)线性得到。

这就证明了正弦位置编码能够通过线性变换来表示相对位置关系。这意味着，在自注意力机制中，模型可以通过学习来利用这种线性关系，从而捕捉相对位置信息。


